# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line)

# creating a smaller event data csv file called event_datafile_new.csv that will be used to insert data into the 
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [3]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now we are ready to work with the CSV file titled *event_datafile_new.csv,* located within the workspace directory.  The *event_datafile_new.csv* contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Apache Cassandra Coding Section

#### Creating a Cluster

In [4]:
# This makes a connection to a Cassandra instance on our local machine
from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])
except Exception as e:
    print(e)

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [5]:
# specify data replication strategy
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class': 'SimpleStrategy', 'replication_factor': 1}"""
    )
except Exception as e:
    print(e)

#### Set Keyspace

In [6]:
# set the current keyspace to work with our database we created
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra we think of queries first!!!

## Create queries to ask the following **three** questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

In [7]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
# query1 = "select artist, song, length where sessionId = 338 and itemInSession = 4"
query1 = "CREATE TABLE IF NOT EXISTS discography "
query1 = query1 + "(artist text, song text, length float, sessionId int, itemInSession int, PRIMARY KEY (sessionId, itemInSession))"

try:
    session.execute(query1)
except Exception as e:
    print(e)

In [8]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO discography (artist, song, length, sessionId, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [9]:
query = "SELECT * FROM discography WHERE sessionId = 338 AND itemInSession = 4"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.length, row.sessionid, row.iteminsession)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875 338 4


### NOW LET'S QUERY BASED TABLE CREATION FOR THE REMAINING TWO QUESTIONS.

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [10]:
# Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS user_artist_relation "
query2 = query2 + "(artist text, song text, firstName text, lastName text, userId int, sessionId int, itemInSession int, PRIMARY KEY ((userId, sessionId), itemInSession))"

try:
    session.execute(query2)
except Exception as e:
    print(e)

In [11]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_artist_relation (userId, sessionId, artist, song, firstName, lastName, itemInSession)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), int(line[8]), line[0], line[9], line[1], line[4], int(line[3])))

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = "SELECT * FROM user_artist_relation WHERE userId = 10 AND sessionId = 182"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname, row.iteminsession)

Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [13]:
query3 = "CREATE TABLE IF NOT EXISTS user_info "
query3 = query3 + "(userId int, firstName text, lastName text, song text, PRIMARY KEY (song, userId))"

try:
    session.execute(query3)
except Exception as e:
    print(e)

In [14]:
# transformation and loading part of the ETL
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:
        
        # Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_info (userId, firstName, lastName, song)"
        query = query + " VALUES (%s, %s, %s, %s)"
        
        # Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[10]), line[1], line[4], line[9]))

#### Do a SELECT to verify that the data have been inserted into each table

In [15]:
query = "SELECT firstName, lastName FROM user_info WHERE song = 'All Hands Against His Own'"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

for row in rows:
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [16]:
# Drop the table before closing out the sessions
tables = ["discography", "user_artist_relation", "user_info"]
queries = ["DROP TABLE {}".format(i) for i in tables]
for query in queries:
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)

### Close the session and cluster connection¶

In [17]:
session.shutdown()
cluster.shutdown()